Evaluating cluster similarity is a bit non-trivial compared to supervised classification algorithms. Check [Scikit-learn documentation](https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation) for more. One way is to compute the *pair*-confusion matrix.

![](./assets/pair-confusion.png)

In [1]:
from sklearn.metrics.cluster import pair_confusion_matrix
from sklearn import metrics
import numpy as np


In [2]:
C = pair_confusion_matrix([0, 0, 1, 1], [0, 0, 1, 2])
C
TN = C[0, 0]
FP = C[0, 1]
FN = C[1, 0]
TP = C[1, 1]

array([[8, 0],
       [2, 2]])

It's easy to compute the Fowlkes–Mallows index, another measure for cluster similarity, using the elements of C as $\text{FMI} = \frac{\text{TP}}{\sqrt{(\text{TP} + \text{FP}) (\text{TP} + \text{FN})}}$. Fowlkes–Mallows index is useful because it gives a number—and not a matrix like the pair confusion matrix—for quickly comparing two clusterings. For more: https://scikit-learn.org/stable/modules/clustering.html#fowlkes-mallows-scores

In [3]:
FMI = TP / np.sqrt((TP + FP) * (TP + FN))
FMI

# You can also compute FMI using scikit-learn. The results match, of course.
metrics.fowlkes_mallows_score([0, 0, 1, 1], [0, 0, 1, 2])

0.7071067811865475

0.7071067811865476

of course, two same clusterings will have no off-diagonal elements in the pair confusion matrix and a FMI score of 1.

In [4]:
C = pair_confusion_matrix([0, 0, 1, 1], [0, 0, 1, 1])
C
TN = C[0, 0]
FP = C[0, 1]
FN = C[1, 0]
TP = C[1, 1]

FMI = TP / np.sqrt((TP + FP) * (TP + FN))
FMI

metrics.fowlkes_mallows_score([0, 0, 1, 1], [0, 0, 1, 1])

array([[8, 0],
       [0, 4]])

1.0

1.0

## Problems with FMI and NMI

Gates et al. raised an objection against FMI and Normalized mutual information in a series of papers (see below). One figure from their paper, reproduced below, summarizes the problem well.

- Gates, A.J., Wood, I.B., Hetrick, W.P. et al. Element-centric clustering comparison unifies overlaps and hierarchy. Sci Rep 9, 8574 (2019). https://doi.org/10.1038/s41598-019-44892-y
- Gates et al., (2019). CluSim: a python package for calculating clustering similarity. Journal of Open Source Software, 4(35), 1264, https://doi.org/10.21105/joss.01264

The have a GitHub repo with their proposed index. Below you can find an example code that reproduces the numbers in the schematic.

- https://github.com/Hoosier-Clusters/clusim

![](./assets/41598_2019_44892_Fig1_HTML.webp)

### Figure from Gates et al. Scientific Reports paper

## The code reproduces the numbers in the schematic (uses their package). The common scores (FMI, NMI etc.) matches if you compute with scikit-learn, too.

In [5]:
from clusim.clustering import Clustering
import clusim.sim as sim

true_labels = [1, 1, 1, 2, 2, 2, 3, 3, 3]
predicted_labels = [1, 2, 2, 3, 3, 1, 1, 1, 1]
single_cluster_labels = [1, 1, 1, 1, 1, 1, 1, 1, 1]
completely_fragmented_labels = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# their data is differently formatted.
true_clustering = Clustering().from_membership_list(true_labels)
predicted_clustering = Clustering().from_membership_list(predicted_labels)
predicted_single_cluster = Clustering().from_membership_list(single_cluster_labels)
predicted_completely_fragmented = Clustering().from_membership_list(
    completely_fragmented_labels
)

for _ in [
    predicted_clustering,
    predicted_single_cluster,
    predicted_completely_fragmented,
]:
    print(
        f"FMI = {sim.fowlkes_mallows_index(true_clustering,_)}, NMI = {sim.nmi(true_clustering,_)}, elem-cent = {sim.element_sim(true_clustering,_)}"
    )

FMI = 0.4811252243246881, NMI = 0.5451600159416435, elem-cent = 0.5407407407407406
FMI = 0.5, NMI = 0.0, elem-cent = 0.33333333333333326
FMI = 0.0, NMI = 0.6666666666666665, elem-cent = 0.33333333333333326


In [6]:
# The package can compute many scores such as... (code from their documentation https://hoosier-clusters.github.io/clusim/html/clusim.html)

row_format2 = "{:>25}" * (2)
for simfunc in sim.available_similarity_measures:
    print(
        row_format2.format(
            simfunc, eval("sim." + simfunc + "(true_clustering, predicted_clustering)")
        )
    )

            jaccard_index                   0.3125
               rand_index       0.6944444444444444
            adjrand_index      0.26666666666666655
    fowlkes_mallows_index       0.4811252243246881
                 fmeasure      0.47619047619047616
             purity_index       0.7777777777777777
     classification_error      0.22222222222222232
        czekanowski_index      0.47619047619047616
               dice_index      0.47619047619047616
           sorensen_index      0.47619047619047616
    rogers_tanimoto_index       0.5319148936170213
          southwood_index      0.45454545454545453
      pearson_correlation      0.00102880658436214
         corrected_chance        0.170353982300885
      sample_expected_sim      0.16666666666666666
                      nmi       0.5451600159416435
                       mi       0.8233232815796736
                   adj_mi       0.3410389011275906
                      rmi       0.1464053299155769
                       vi      